# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770944328244                   -0.52    9.0         
  2   -2.772148827956       -2.92       -1.32    1.0    154ms
  3   -2.772170343038       -4.67       -2.46    1.0    194ms
  4   -2.772170659401       -6.50       -3.17    1.0    160ms
  5   -2.772170722502       -7.20       -4.07    2.0    180ms
  6   -2.772170722745       -9.61       -4.27    1.0    197ms
  7   -2.772170723012       -9.57       -5.85    1.0    166ms
  8   -2.772170723015      -11.59       -6.43    2.0    206ms
  9   -2.772170723015   +  -13.58       -6.19    2.0    193ms
 10   -2.772170723015      -13.32       -6.98    1.0    203ms
 11   -2.772170723015      -14.40       -7.40    1.0    174ms
 12   -2.772170723015   +  -13.94       -8.43    1.0    185ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.7

1.7735580538164621

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770883150621                   -0.53    9.0         
  2   -2.772062331869       -2.93       -1.32    1.0    154ms
  3   -2.772083170252       -4.68       -2.50    1.0    206ms
  4   -2.772083363098       -6.71       -3.25    1.0    159ms
  5   -2.772083415305       -7.28       -3.83    2.0    226ms
  6   -2.772083417698       -8.62       -4.70    1.0    163ms
  7   -2.772083417808       -9.96       -5.26    2.0    188ms
  8   -2.772083417810      -11.58       -5.93    1.0    179ms
  9   -2.772083417811      -12.51       -6.30    2.0    214ms
 10   -2.772083417811      -13.51       -7.71    1.0    183ms
 11   -2.772083417811   +  -14.45       -8.19    2.0    205ms

Polarizability via ForwardDiff:       1.7725349719820247
Polarizability via finite difference: 1.7735580538164621
